# Dependencies

In [ ]:
! pip -q install datasets evaluate transformers rouge-score nltk accelerate -U ir-datasets rank_bm25
! pip -q install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 14.4 MB/s eta 0:00:00
  

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_metric
import random
from pathlib import Path

import warnings
warnings.simplefilter("ignore")

# Notebook login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load Dataset

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("satyanshu404/trec-cast-2019")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'description', 'title', 'prompts'],
        num_rows: 30
    })
    test: Dataset({
        features: ['number', 'description', 'title', 'prompts'],
        num_rows: 50
    })
})

In [ ]:
raw_dataset['test'][0]

{'number': 31,
 'description': 'A person is trying to compare and contrast types of cancer in the throat, esophagus, and lungs.',
 'title': 'head and neck cancer',
 'prompts': "What is throat cancer?, Is it treatable?, Tell me about lung cancer., What are its symptoms? , Can it spread to the throat?, What causes throat cancer?, What is the first sign of it?, Is it the same as esophageal cancer?, What's the difference in their symptoms?"}

In [ ]:
def length(textlist):
    import numpy as np
    lengt = []
    for text in textlist:
        if text == None:
            lengt.append(0)
            continue
        lengt.append(len(text.split(" ")))
    return np.mean(lengt)

print(length(raw_dataset['train']['description']))

12.966666666666667


# Model Checkpoints

In [ ]:
model_checkpoint = "facebook/bart-large-cnn"
# model_checkpoint = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3"

# Preprocessing and Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
column1 = 'description' # input
column2 = 'prompts'  # output

In [ ]:
prefix = "Based on the given query generate appropriate prompts that are relavent to the given query."

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [f"Instruction: {prefix}\n Query: {doc}" for doc in examples[column1]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples[column2], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['number', 'description', 'title', 'prompts', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30
    })
    test: Dataset({
        features: ['number', 'description', 'title', 'prompts', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})

In [ ]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

# Metric

In [ ]:
from evaluate import load
metric = load("rouge")

In [ ]:
# def bm25_score(corpus, query):
#     from rank_bm25 import BM25Okapi
#     import numpy as np

#     tokenized_corpus = [doc.split(" ") for doc in corpus]

#     bm25 = BM25Okapi(tokenized_corpus)
#     tokenized_query = query.split(" ")
#     doc_scores = bm25.get_scores(tokenized_query)

#     return doc_scores

In [ ]:
# def find_scores(querylist, corpuslist):
#     import numpy as np
#     scores = []
#     for query, sentence in zip(querylist, corpuslist):
#         corpus = sentence.split(",")
#         if query is None:
#             scores.append(np.array([0.0]*len(corpus)))
#             continue
#         scores.append(bm25_score(corpus, query))
#     return scores

In [ ]:
# def apk(prediction):
#     from scipy.special import softmax
#     score = 0.0
#     relevant_document = 0.0

#     prediction = softmax(prediction)

#     for i, predicted in enumerate(prediction):
#         if predicted > 0.5:
#             relevant_document += 1.0
#             score += relevant_document / (i + 1.0)

#     if not relevant_document:
#         return 0.0
#     return score / relevant_document

# def mapk(predictions):
#     map_scores = []
#     for prediction in predictions:
#         map_scores.append(apk(prediction))

#     return np.mean(map_scores)

In [ ]:
# import numpy as np

# def dcg_score(prediction, idcg=False, k=10):
#     from scipy.special import softmax
#     score = 0.0

#     if idcg:
#         prediction = sorted(prediction, reverse=True)

#     prediction = softmax(prediction)

#     if len(prediction) > k:
#         prediction = prediction[:k]



#     for i, predicted in enumerate(prediction):
#         if predicted > 0.5:
#             score += (1) / np.log2(i + 2.0)
#     return score


# def ndcg_score(predictions, k=10):
#     total_score = []

#     for prediction in predictions:
#         dcg = dcg_score(prediction, idcg=False, k=10)
#         idcg = dcg_score(prediction, idcg=True, k=10)
#         # ndcg score
#         if not idcg:
#             total_score.append(0.0)
#         else:
#             total_score.append(dcg/idcg)

#     return np.mean(total_score)

In [ ]:
import nltk

def compute_metrics(eval_pred, input):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # preds_scores = find_scores(input, decoded_preds)
    # labels_scores = find_scores(input, decoded_labels)

    # y_score = [np.array(pred, dtype=float) for pred in preds_scores]

    # map_score = mapk(y_score)
    # NDCG_score = ndcg_score(y_score, 10)

    # actual_score = np.mean([np.mean(list_) for list_ in labels_scores])
    # predction_score = np.mean([np.mean(list_) for list_ in preds_scores])

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)


    return {k: round(v, 4) for k, v in result.items()}


# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-prompt_generation",
    evaluation_strategy = "epoch",
    learning_rate=3e-7,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=lambda x: compute_metrics(x, tokenized_datasets["test"]['description'])
)

In [ ]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [ ]:
trainer.train()

wandb: Currently logged in as: satyanshu (satyanshu404). Use `wandb login --relogin` to force relogin


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.656215,25.090300,5.315800,16.426500,16.385300,67.420000
2,No log,3.553926,24.901100,4.985400,16.581200,16.569700,65.280000
3,No log,3.393049,24.998300,5.237300,17.034200,16.993000,65.800000
4,No log,3.292787,24.841800,4.715900,16.929000,16.907000,66.000000
5,No log,3.172303,26.012000,5.569600,17.400200,17.462100,66.840000
6,No log,3.081317,26.944300,5.826200,17.829700,17.867300,67.520000
7,No log,3.016928,27.715500,6.429700,18.447900,18.491300,66.780000
8,No log,2.969966,27.285800,6.543700,18.518500,18.473100,67.780000
9,No log,2.933992,28.074700,7.304900,18.704500,18.718000,67.340000
10,No log,2.904366,28.441700,7.340000,18.780500,18.837700,66.440000


TrainOutput(global_step=750, training_loss=2.541506591796875, metrics={'train_runtime': 3084.7124, 'train_samples_per_second': 0.486, 'train_steps_per_second': 0.243, 'total_flos': 148544015646720.0, 'train_loss': 2.541506591796875, 'epoch': 50.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.6454074382781982,
 'eval_rouge1': 40.6908,
 'eval_rouge2': 16.1706,
 'eval_rougeL': 25.6927,
 'eval_rougeLsum': 25.6588,
 'eval_gen_len': 77.2,
 'eval_runtime': 61.9037,
 'eval_samples_per_second': 0.808,
 'eval_steps_per_second': 0.404,
 'epoch': 50.0}

# Upload on Hub

In [ ]:
!apt install git-lfs
trainer.push_to_hub()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1700938408.28f3f5ce4d66.281.1:   0%|          | 0.00/613 [00:00<?, ?B/s]

events.out.tfevents.1700935261.28f3f5ce4d66.281.0:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

'https://huggingface.co/satyanshu404/bart-large-cnn-prompt_generation/tree/main/'

# Inference

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("satyanshu404/bart-large-cnn-finetuned-prompt_generation")
model_trained = AutoModelForSeq2SeqLM.from_pretrained("satyanshu404/bart-large-cnn-finetuned-prompt_generation")

In [ ]:
prompt = """
Instruction: Based on the given description generate a list of all the prompts that are relavent to the document and sort the prompts in decreasing relevance.
Query: The history of the toilet and the evolution of indoor and outdoor sanitation systems.
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
gen_tokens = model_trained.generate(input_ids, temperature=0.7)
gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(gen_text)

What is the history of the toilet and the evolution of indoor and outdoor sanitation systems? What is the difference between a toilet and a urinal? What are the different types of toilets? How do they differ from each other? What do they look like? How are they different?


In [ ]:
prompt = """
Instruction: Based on the given description generate a list of all the prompts that are relavent to the document and sort the prompts in decreasing relevance.
Query: The history of the toilet and the evolution of indoor and outdoor sanitation systems.
"""

In [ ]:
Generated:

# The history of the toilet and the evolution of indoor and outdoor sanitation systems.
# The evolution of toilets in the U.S. and other parts of the world.
# The history of toilets and how they were used in the United States.
# How they were first used.
# What was the first toilet used for?
# What was its purpose?

In [ ]:
Actual:

# Tell me about the history of toilets.
# Where does the term come from?
# Where and when was the first invented?
# Why do the Brits call it a loo?
# What came before them?
# When did indoor versions become common?
# How many types are there?
# Why are they so important for society?
# What is their role in film?